In [1]:
import pandas as pd
from env import get_url_for_pancakes

# Aqcuisition

The data is stored on a sql database. We'll use an env.py with the credintials needed to pull the data from sql. We'll pull the data into a pandas dataframe, which will be the primary datastructure for housing our data.

The sql query itself has some interesting criteria here as well: 


In [2]:
query = """
SELECT  bedroomcnt, 
		bathroomcnt,
		calculatedfinishedsquarefeet,
		taxvaluedollarcnt,
		fips
FROM properties_2017 as p
JOIN predictions_2017 as pr USING(`parcelid`)
WHERE propertylandusetypeid = 261 and (transactiondate >= '2017-05-01' AND transactiondate <= '2017-06-31')
"""

zillow_url = get_url_for_pancakes('zillow')

df = pd.read_sql(query, zillow_url)

In [3]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,fips
0,3.0,2.0,1458.0,136104.0,6037.0
1,2.0,1.0,1421.0,35606.0,6037.0
2,4.0,3.0,2541.0,880456.0,6059.0
3,3.0,2.0,1650.0,614000.0,6037.0
4,2.0,1.0,693.0,274237.0,6037.0


In [4]:
df.shape

(15036, 5)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15036 entries, 0 to 15035
Data columns (total 5 columns):
bedroomcnt                      15036 non-null float64
bathroomcnt                     15036 non-null float64
calculatedfinishedsquarefeet    15012 non-null float64
taxvaluedollarcnt               15036 non-null float64
fips                            15036 non-null float64
dtypes: float64(5)
memory usage: 587.5 KB


In [14]:
df.fips = df.fips.astype(int)

In [6]:
df.isna().sum()

bedroomcnt                       0
bathroomcnt                      0
calculatedfinishedsquarefeet    24
taxvaluedollarcnt                0
fips                             0
dtype: int64

In [15]:
fips = pd.read_clipboard()

In [18]:
fips.head()

,FIPS,Name,State
0,1001,Autauga,AL
1,1003,Baldwin,AL
2,1005,Barbour,AL
3,1007,Bibb,AL
4,1009,Blount,AL


In [20]:
df[df.fips IN fips.FIPS]

SyntaxError: invalid syntax (<ipython-input-20-a3a1a5bf51c4>, line 1)

# Preparation

## Clean the Data

In [7]:
df.columns

Index(['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet',
       'taxvaluedollarcnt', 'fips'],
      dtype='object')

In [8]:
df.rename(columns={'bedroomcnt':'bedroom_count', 'bathroomcnt':'bathroom_count', 'calculatedfinishedsquarefeet':'total_sqft',
       'taxvaluedollarcnt':'tax_amount'}, inplace=True)

In [9]:
df.head(1)

,bedroom_count,bathroom_count,total_sqft,tax_amount,fips
0,3.0,2.0,1458.0,136104.0,6037.0


# Exploration

# Feature Engineering

In [10]:
# 